In [39]:
import pandas as pd
import numpy as np
import nltk
nltk.download('wordnet')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer 
import re
import sys
sys.path.append('../resume_parser/')
import resume_parser
# from resume_parser import resume_parser

jd_path = "../data/JD/Software Engineer Intern in ML Engineering Platform (System) - 2021 Summer - ByteDance.pdf"
resume_path = "../data/resume/ivan_machine_learning_engineer.pdf"
job_description = resume_parser.get_text(jd_path)
resume = resume_parser.get_text(resume_path)



[nltk_data] Downloading package wordnet to /home/yk09/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/yk09/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
import os
os.listdir("../data/resume")

['Chang_Ker_Fui.docx',
 'Chang_Ker_Fui_Resume.pdf',
 'ivan_machine_learning_engineer.pdf',
 'Jaryl Lim Resume.docx',
 'machine_learning_engineer_airbnb.pdf',
 'resumes.zip',
 'Ruolin_zhang_psych.pdf',
 'sample_input.pdf',
 'sample_output.txt',
 'testdata.json',
 'traindata.json',
 'train_data.pkl']

In [16]:


# A list of text
def clean_text(text):
    text = re.sub(r'[0-9]', '',text).replace(u'\xa0', u' ')
    return text


resume,job_description = clean_text(resume),clean_text(job_description)
text = [resume, job_description]


predefined_stop_words = ["the","a","is","you","your","will"]

stop_words = list(set(stopwords.words('english'))) + predefined_stop_words


# Methods

## Baseline method with cosine similarity and Count Vectorizer

In [5]:
# A list of text


import nltk
nltk.download('wordnet')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer 

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

stop_words = list(set(stopwords.words('english'))) + predefined_stop_words

class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, articles):
        return [self.wnl.lemmatize(t) for t in word_tokenize(articles)]

cv = CountVectorizer(analyzer=clean_text,tokenizer=LemmaTokenizer(),stop_words=stop_words)


# Lemma and stopwords
print(resume_path)
cv = CountVectorizer(tokenizer=LemmaTokenizer(),stop_words=stop_words)
# generates word counts for the words in resume and jd 
count_matrix = cv.fit_transform(text)
#Print the similarity scores
print("\nSimilarity Scores Lemma and stopwords:")
print(cosine_similarity(count_matrix))

# # Lemma without stop words
# cv = CountVectorizer(tokenizer=LemmaTokenizer())
# count_matrix = cv.fit_transform(text)
# #Print the similarity scores
# print("\nSimilarity Scores Lemma without stop words:")
# print(cosine_similarity(count_matrix))

# # Stop words without lemma
# cv = CountVectorizer(stop_words=stop_words)
# count_matrix = cv.fit_transform(text)
# #Print the similarity scores
# print("\nSimilarity Scores Lemma without stop words:")
# print(cosine_similarity(count_matrix))

# # without lemma and stop words
# cv = CountVectorizer()
# count_matrix = cv.fit_transform(text)
# #Print the similarity scores
# print("\nSimilarity Scores without lemma and stop words:")
# print(cosine_similarity(count_matrix))


[nltk_data] Downloading package wordnet to /home/yk09/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/yk09/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
../data/resume/ivan_machine_learning_engineer.pdf

Similarity Scores Lemma and stopwords:
[[1.       0.625759]
 [0.625759 1.      ]]

Similarity Scores Lemma without stop words:
[[1.         0.69737129]
 [0.69737129 1.        ]]

Similarity Scores Lemma without stop words:
[[1.         0.26869837]
 [0.26869837 1.        ]]

Similarity Scores without lemma and stop words:
[[1.         0.66345316]
 [0.66345316 1.        ]]


In [6]:
doc_term_matrix = count_matrix.todense()
df = pd.DataFrame(doc_term_matrix, 
                        columns=cv.get_feature_names(), 
                        index=['resume', 'job_description'])

## TFIDF

In [8]:
# from sklearn.feature_extraction.text import TfidfVectorizer,TfidfTransformer
# tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True) 
# tfidf_transformer.fit(count_matrix)
# # print idf values
# df_idf = pd.DataFrame(tfidf_transformer.idf_, index=cv.get_feature_names(),columns=["idf_weights"]) 
 
# # sort ascending 
# df_idf.sort_values(by=['idf_weights'])

# # count matrix 
# count_vector=cv.transform(resume) 
 
# # tf-idf scores 
# tf_idf_vector=tfidf_transformer.transform(count_vector)

## Skill Based Matching

In [17]:
skills_candidate = resume_parser.extract_skills(resume)
skills_jd = resume_parser.extract_skills(job_description)


### Method 1 : Raw Skill Matching

In [12]:
skills = [' '.join(skills_jd),' '.join(skills_candidate)]
cv = CountVectorizer()
skills_matrix = cv.fit_transform(skills)
#Print the similarity scores
print("\nSimilarity Scores without lemma and stop words:")
print(cosine_similarity(skills_matrix))
threshold = 0.2

# this one is lower


Similarity Scores without lemma and stop words:
[[1.         0.30123762]
 [0.30123762 1.        ]]


### Method 2 : Matching by training vectorizer first. Finding the distance of the resume from the job parser

In [38]:
skills_candidate[9:12]
skills_jd[10]

count = 0
for i in skills_candidate:
    for j in skills_jd:
        if i ==j:
            count+=1
            print(i,j)
percentage_match = count/len(skills_jd)
print(percentage_match)

System System
Communication Communication
Tensorflow Tensorflow
Computer science Computer science
Engineering Engineering
Training Training
Process Process
Programming Programming
0.4


In [33]:
def vect_cos(vect, resume_list):
    """ Vectorise text and compute the cosine similarity """
    jd = vect.transform([' '.join(vect.get_feature_names())])
    # print("jd"+str(jd))
    resume_vect = vect.transform(resume_list)
    # print("resume"+str(resume_vect))
    cos_sim = cosine_similarity(jd.A, resume_vect.A)  # displays the resulting matrix
    return resume_vect, np.round(cos_sim.squeeze(), 3)

vectoriser = CountVectorizer().fit(skills_jd)

# Analyze candidate
resume_vect, resume_cos = vect_cos(vectoriser, [' '.join(skills_candidate)])
print('\nThe cosine similarity for the resume is {}.'.format(resume_cos))

# turning it into df
pd.DataFrame(resume_vect.toarray())


The cosine similarity for the resume is 0.662.


AttributeError: get_feature_names not found

### Method 3 : Using Counter

In [15]:
from collections import Counter

# count word occurrences
resume_vals = Counter(skills_candidate)
jd_vals = Counter(skills_jd)

# convert to word-vectors
words  = list(resume_vals.keys() | jd_vals.keys())
resume_vect = [resume_vals.get(word, 0) for word in words]
jd_vect = [jd_vals.get(word, 0) for word in words]        # [1, 1, 1, 0, 1, 0]

# find cosine
len_r  = sum(rv*rv for rv in resume_vect) ** 0.5             # sqrt(7)
len_jd  = sum(jdv*jdv for jdv in jd_vect) ** 0.5             # sqrt(4)
dot    = sum(rv*jdv for rv,jdv in zip(resume_vect, jd_vect))    # 3
cosine = dot / (len_r * len_jd)
print('The cosine similarity for the resume is {}.'.format(cosine))
                         

[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1] [1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0]


NameError: name 'len_a' is not defined

## Matching Education